# Using pre-trained network with Chainer: classification and visualization

We'll classify an image with ResNet 152 pretrained model.
We'll compare CPU and GPU modes and then dig into the model to inspect features and the output.

### 1. Setup

In [1]:
import chainer, tqdm, os, cv2, random
import numpy as np
from skimage import io
from scipy.misc import imresize
from matplotlib import pyplot as plt
from time import time

print('Chainer version: {}'.format(chainer.__version__))

# Function for visualization of classification outpus
def show_picture_and_probability(img, probability, labels, howmany=5):
    plt.rcParams["figure.figsize"] = [6.4, 4.8]
    plt.clf()
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.subplot(1, 2, 2)
    plt.subplots_adjust(left=0.1, right=2.5, bottom=0.1, top=0.9)
    x = list(range(1, howmany + 1))
    y = []
    ticks =[]
    for i in np.argsort(probability)[::-1][: howmany]:
        y.append(probability[i])
        ticks.append(' ,'.join(labels[i].split(',')[:3]))
    plt.barh(x, y[::-1], align="center") 
    plt.yticks(x, ticks[::-1]) 
    plt.show()

# Function for visualization of activations
def show_activations(img, activations, howmany=5):
    plt.rcParams["figure.figsize"] = [100, 100]
    plt.clf()
    plt.subplot(1,howmany + 1, 1)
    plt.axis('off')
    plt.imshow(img)
    for i in range(2, howmany + 1 + 1):
        plt.subplot(1,howmany + 1, i)
        plt.axis('off')
        plt.imshow(cv2.resize(activations[i], (224, 224)))
    plt.show()

# aspect ratio is kept after resizing
def resize_image(img, minimum_length=256):
        y, x = img.shape[:2]
        # keep aspect ratio
        if y <= x:
            scale = float(minimum_length) / y
            sizes = (minimum_length, int(scale * x))
        else:
            scale = float(minimum_length) / x
            sizes = (int(scale * y), minimum_length)
        return imresize(img, sizes, interp='bilinear', mode='RGB')

# crop pictures at center
def crop_center(img, sizes=(224, 224)):
        y, x, channel = img.shape
        center_y, center_x = int(y / 2), int(x / 2)
        frame_y, frame_x = sizes
        up, down = -int((frame_y + 1) / 2), int(frame_y / 2)
        left, right = -int((frame_x + 1) / 2), int(frame_x / 2)
        start_h, end_h = max(center_y + up, 0), min(center_y + down, y)
        start_w, end_w = max(center_x + left, 0), min(center_x + right, x)
        return img[start_h:end_h, start_w:end_w]


Chainer version: 2.1.0


### 2. Load ImageNet Pretrained ResNet 152

In [3]:
# prepare label information
labels = []
with open('/root/userspace/readonly/chapter1/imagenet_labels.txt', 'r') as f:
    line = f.readline()
    while line:
                labels.append(line[:-1])
                line = f.readline()
# define ResNet and load weights
model = chainer.links.model.vision.resnet.ResNetLayers('/root/userspace/readonly/chapter1/resnet_152.caffemodel',  152)

### 3. CPU classification
* Now we're ready to perform classification. Even though we'll only classify one image, we'll set a batch size of 10 to demonstrate batching.

In [ ]:
from matplotlib import pyplot as plt

# Batch size
batch = 10

# Send to cpu
model.to_cpu()

# Find pictures
files = ['/root/userspace/readonly/chapter1/example_pictures/{}'.format(name) for name in os.listdir('/root/userspace/readonly/chapter1/example_pictures')]

# Load pictures randomly
random.shuffle(files)
imgs = []
for i in range(batch):
    imgs.append(crop_center(resize_image(io.imread(files[np.random.randint(len(files))]), 256)))
x = np.array(imgs, dtype=np.float32)

# Forward
print('start forwarding')
start = time()
with chainer.using_config('train', False):
    y = model.extract(x,  layers=['prob'])
end = time()
print('    done')
print('Total execution time: {} seconds'.format(end - start))
print('Execution time per picture: {} seconds'.format((end - start) / batch))

# Visualize
show_picture_and_probability(imgs[0], y['prob'].data[0], labels, howmany=5)
# Visualize
show_picture_and_probability(imgs[1], y['prob'].data[1], labels, howmany=5)
# Visualize
show_picture_and_probability(imgs[2], y['prob'].data[2], labels, howmany=5)
# Visualize
show_picture_and_probability(imgs[3], y['prob'].data[3], labels, howmany=5)
# Visualize
show_picture_and_probability(imgs[4], y['prob'].data[4], labels, howmany=5)

### 4. Switching to GPU mode
* Let's see how long classification take and compare it to GPU mode.

In [ ]:
from matplotlib import pyplot as plt

# Batch size
batch = 10

# Send to cpu
model.to_gpu()

# Find pictures
files = ['/root/userspace/readonly/chapter1/example_pictures/{}'.format(name) for name in os.listdir('/root/userspace/readonly/chapter1/example_pictures')]

# Load pictures randomly
random.shuffle(files)
imgs = []
for i in range(batch):
    imgs.append(crop_center(resize_image(io.imread(files[np.random.randint(len(files))]), 256)))
x = np.array(imgs, dtype=np.float32)

# Forward
print('start forwarding')
start = time()
with chainer.using_config('train', False):
    y = model.extract(x,  layers=['prob'])
end = time()

print('    done')
print('Total execution time: {} seconds'.format(end - start))
print('Execution time per picture: {} seconds'.format((end - start) / batch))

# send back to cpu
y['prob'].to_cpu()
model.to_cpu()

# Visualize
show_picture_and_probability(imgs[0], y['prob'].data[0], labels, howmany=5)
# Visualize
show_picture_and_probability(imgs[1], y['prob'].data[1], labels, howmany=5)
# Visualize
show_picture_and_probability(imgs[2], y['prob'].data[2], labels, howmany=5)
# Visualize
show_picture_and_probability(imgs[3], y['prob'].data[3], labels, howmany=5)
# Visualize
show_picture_and_probability(imgs[4], y['prob'].data[4], labels, howmany=5)

### 5. Examining intermediate output

* Neural Networks are not just a black box; let's take a look at some of the parameters and intermediate activations.

First we'll see how to read out the structure of the net in terms of activation and parameter shapes.

* For each layer, let's look at the activation shapes, which typically have the form `(batch_size, channel_dim, height, width)`.

In [ ]:
# Batch size
batch = 1

# Send to cpu
model.to_gpu()

# Find pictures
files = ['/root/userspace/readonly/chapter1/example_pictures/{}'.format(name) for name in os.listdir('/root/userspace/readonly/chapter1/example_pictures')]

# Load pictures randomly
imgs = []
for _ in range(batch):
    imgs.append(imresize(io.imread(files[np.random.randint(len(files))]), (256, 256) , interp='bilinear', mode='RGB'))

# Shape is (1, 3, 224, 224) -> (batch, channel, height, width)
x = np.array(imgs, dtype=np.float32)

# Forward
print('start forwarding')
start = time()
with chainer.using_config('train', False):
    y = model.extract(x,  layers=['conv1', 'pool1', 'res2', 'res3', 'res4', 'res5', 'pool5', 'fc6', 'prob'])
end = time()

# Send back to cpu
[y[key].to_cpu() for key in y]
model.to_cpu()

print('    done')
print('Total execution time: {} seconds'.format(end - start))
print('Execution time per picture: {} seconds'.format((end - start) / batch))
print('####################################')
print("Let's see activation maps:")

print('conv1 shape: {}'.format(y['conv1'][0].data.shape))
show_activations(imgs[0], y['conv1'][0].data)

print('pool1 shape: {}'.format(y['pool1'][0].data.shape))
show_activations(imgs[0], y['pool1'][0].data)

print('res2 shape: {}'.format(y['res2'][0].data.shape))
show_activations(imgs[0], y['res2'][0].data)

print('res3 shape: {}'.format(y['res3'][0].data.shape))
show_activations(imgs[0], y['res3'][0].data)

print('res4 shape: {}'.format(y['res4'][0].data.shape))
show_activations(imgs[0], y['res4'][0].data)

print('res5 shape: {}'.format(y['res5'][0].data.shape))
show_activations(imgs[0], y['res5'][0].data)